# Importing Necessary library

In [1]:
import os
import gc
import vtk
import cv2
import time
import pydicom
import numpy as np 
import pandas as pd 
import scipy.ndimage
import seaborn as sns
import tensorflow as tf
import pandas as pd
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go

from glob import glob
from skimage import measure
from tensorflow import keras
from plotly import __version__
from plotly.graph_objs import*
from skimage import morphology
from vtk.util import numpy_support
from sklearn.cluster import KMeans
from skimage.transform import resize
from IPython.display import clear_output
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from plotly.tools import FigureFactory as FF
from tensorflow.keras.models import load_model
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from tensorflow.keras.callbacks import ModelCheckpoint as MC
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


# Data path setup

In [2]:
root_path ='../input/rsna-str-pulmonary-embolism-detection'
for item in os.listdir(root_path):
    path = os.path.join(root_path, item)
    if os.path.isfile(path):
        print(path)
print("Reading the training dataset")
train = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
print(train.shape)
train.head(4)

In [3]:
print('Reading test data...')
test = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/test.csv")
print(test.shape)
test.head(4)

In [4]:
train.info()

In [5]:
test.info()

# Reading Dataset

In [6]:
train_path = "../input/rsna-str-pulmonary-embolism-detection/train"
class_names = os.listdir(train_path)

In [7]:
print(len(class_names))

In [ ]:
sns.histplot(test.SOPInstanceUID, kde=True)
plt.show()

In [ ]:

sns.heatmap(test.corr(), annot=True, fmt=".2f")
plt.show()

Histogram Binning

In [8]:
from skimage import io
import matplotlib.pyplot as plt
h_path="../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf/00ac73cfc372.dcm"
image = io.imread(h_path)
plt.hist(image.ravel(), bins = 40 )
plt.xlabel('Intensity Value')
plt.ylabel('Count')
plt.show()
plt.savefig("Binned Histogram-with 40 bins.jpg")

Cumulative Histogram

In [9]:
from skimage import io
import matplotlib.pyplot as plt
image = io.imread(h_path)
plt.hist(image.ravel(), bins = 256, cumulative = True)
plt.xlabel('Intensity Value')
plt.ylabel('Count') 
plt.show()
plt.savefig("Cumulative Histogram.jpg")

Grayscale  Histogram

In [10]:
from skimage import io
import matplotlib.pyplot as plt
image = io.imread(h_path)
plt.hist(image.ravel(), bins = 200)
plt.xlabel("Intensity")
plt.ylabel("Count")
plt.show()
plt.savefig("grayscale_img_histogram.jpg")

# Preprocessing

In [11]:
import pydicom
file_path = h_path
medical_image = pydicom.read_file(file_path)
print(medical_image)

In [12]:
image = medical_image.pixel_array
print(image.shape)
plt.imshow(image,cmap='gray')
print(image.min())
print(image.max())
plt.savefig("reshaped.jpg")

# Importing Dicom file

In [13]:
reader = vtk.vtkDICOMImageReader()
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]
    ConstPixelSpacing = reader.GetPixelSpacing()
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims,order='F')
    ArrayDicom = cv2.resize(ArrayDicom,(512,512))
    return ArrayDicom
data_path ="../input/rsna-str-pulmonary-embolism-detection/train/00511e94edec/297f170f1197"
def load_scan(path):
    slices = [pydicom.read_file(path+'/'+ s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2]-slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation-slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

def get_pixels(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    image[image == -2000] = 0
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    if slope != 1:
        image = slope*image.astype(np.float64)
        image = image.astype(np.int16)
    image += np.int16(intercept)
    return np.array(image,dtype=np.int16)
id=0
patient = load_scan(data_path)
imgs = get_pixels(patient)
output_dir = working_dir = "./"
np.save(output_dir + "fullimages_%d.npy"%(id),imgs)
import matplotlib.pyplot as plt
file_used=output_dir+"fullimages_%d.npy"%id
imgs_to_process = np.load(file_used).astype(np.float64) 

plt.hist(imgs_to_process.flatten(), bins=20, color='blue')
plt.xlabel("Hounsfield Units")
plt.ylabel("Frequency")
plt.show()
plt.savefig("ONE.jpg")

# Reading Dicom files

In [14]:
id = 0
imgs_to_process = np.load(output_dir+'fullimages_{}.npy'.format(id))
def sample_stack(stack, rows=5, cols=5, start_with=10, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[20,20])
    for i in range(rows*cols):
        index = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('Image-%d'%index)
        ax[int(i/rows),int(i % rows)].imshow(stack[index],cmap='YlOrRd')
        ax[int(i/rows),int(i % rows)].axis('on')
    plt.show()
    plt.savefig("two.jpg")
sample_stack(imgs_to_process)

In [15]:
id = 0
imgs_to_process = np.load(output_dir+'fullimages_{}.npy'.format(id))
def resample(image, scan, new_spacing=[2,2,2]):
    spacing = map(float, ([scan[0].SliceThickness]+list(scan[0].PixelSpacing)))
    spacing = np.array(list(spacing))
    resize_factor = spacing/new_spacing
    new_real_shape = image.shape*resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape/image.shape
    new_spacing = spacing/real_resize_factor
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    return image, new_spacing
print("Shape before resampling =", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1,1,1])
print("Shape after resampling =", imgs_after_resamp.shape)

# Defining Image Reader 

In [16]:
def make_mesh(image, threshold=-200, step_size=3):
    print( "Transposing surface")
    p = image.transpose(2,1,0)
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces
def plotly_3d(verts, faces):
    x,y,z = zip(*verts) 
    print("Drawing") 
    colormap=['rgb(20,0, 5)','rgb(0,0,255)']
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor='rgb(1,22,1)')
    iplot(fig)
def plt_3d(verts,faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_facecolor((0.5, 0.2, 0.4))
    plt.show()

In [17]:
v, f = make_mesh(imgs_after_resamp, 300)
plt_3d(v, f)
plt.savefig("meshed.jpg")

# Masking the unnecessary pixels

In [18]:
def make_lungmask(img,display=False):
    row_size= img.shape[0]
    col_size = img.shape[1]
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    #pixel value
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    img[img==max]=mean
    img[img==min]=mean
    # Using Kmeans to separate foreground 
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.5,0.0)
    # First erode away
    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))
    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0
    # lung mask 
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10]))#last dialation
    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[15,15])
        ax[0, 0].set_title("Original-Image")
        ax[0, 0].imshow(img, cmap='YlOrRd')
        ax[0, 0].axis('on')
        ax[0, 1].set_title("Threshold-Image")
        ax[0, 1].imshow(thresh_img, cmap='YlOrRd')
        ax[0, 1].axis('on')
        ax[1, 0].set_title("Image-After-Erosion")
        ax[1, 0].imshow(dilation, cmap='YlOrRd')
        ax[1, 0].axis('on')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('on')
        ax[2, 0].set_title("Masked-Image")
        ax[2, 0].imshow(mask, cmap='YlOrRd')
        ax[2, 0].axis('on')
        ax[2, 1].set_title("Mask-on-original-img")
        ax[2, 1].imshow(mask*img, cmap='YlOrRd')
        ax[2, 1].axis('on')
        plt.show()
    return mask*img
img = imgs_after_resamp[220]
make_lungmask(img,display=True)

# Masked Image

In [19]:
masked_lung = []
for img in imgs_after_resamp:
    masked_lung.append(make_lungmask(img))
sample_stack(masked_lung, show_every=5)
np.save(output_dir + "maskedimages_%d.npy" % (id), imgs)

# The model

In [20]:
base_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,data_format=K.image_data_format())

In [21]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
   
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def ResNet50(input_shape=(224,224,3)):
    

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(inputs)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=inputs, outputs=X, name='ResNet50')

    return model

In [ ]:
base_model = ResNet50()

In [ ]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 1,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)


In [ ]:
model = Model(inputs=base_model.input, outputs=headModel)
model.summary()

# Model architecture

In [22]:
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.layers import Dropout

inputs = Input((512, 512, 3))
base_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet"
)


base_model.trainable = True

outputs = base_model(inputs, training=True)
outputs = keras.layers.GlobalAveragePooling2D()(outputs)
outputs = Dropout(0.25)(outputs)
outputs = Dense(512, activation='relu')(outputs)
outputs = Dense(256, activation='relu')(outputs)
outputs = Dense(128, activation='relu')(outputs)
outputs = Dense(64, activation='relu')(outputs)
outputs = Dense(32, activation='relu')(outputs)

ppoi = Dense(1, activation='sigmoid', name='pe_present_on_image')(outputs)
rlrg1 = Dense(1, activation='sigmoid', name='rv_lv_ratio_gte_1')(outputs)
rlrl1 = Dense(1, activation='sigmoid', name='rv_lv_ratio_lt_1')(outputs) 
lspe = Dense(1, activation='sigmoid', name='leftsided_pe')(outputs)
cpe = Dense(1, activation='sigmoid', name='chronic_pe')(outputs)
rspe = Dense(1, activation='sigmoid', name='rightsided_pe')(outputs)
aacpe = Dense(1, activation='sigmoid', name='acute_and_chronic_pe')(outputs)
cnpe = Dense(1, activation='sigmoid', name='central_pe')(outputs)
indt = Dense(1, activation='sigmoid', name='indeterminate')(outputs)

model = Model(inputs=inputs, outputs={'pe_present_on_image':ppoi,
                                      'rv_lv_ratio_gte_1':rlrg1,
                                      'rv_lv_ratio_lt_1':rlrl1,
                                      'leftsided_pe':lspe,
                                      'chronic_pe':cpe,
                                      'rightsided_pe':rspe,
                                      'acute_and_chronic_pe':aacpe,
                                      'central_pe':cnpe,
                                      'indeterminate':indt})

opt = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary

In [23]:
model.summary()
model.save('pe_detection_model.h1')

# Model Training

In [24]:
def convert_to_rgb(array):
    array = array.reshape((512, 512, 1))
    return np.stack([array, array, array], axis=2).reshape((512, 512, 3))
def custom_dcom_image_generator(batch_size, dataset, test=False, debug=False):
    fnames = dataset[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']]
    if not test:
        Y = dataset[['pe_present_on_image', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe',
                     'chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate'
                    ]]
        prefix = '../input/rsna-str-pulmonary-embolism-detection/train'

    else:
        prefix = '../input/rsna-str-pulmonary-embolism-detection/test'
    X = []
    batch = 0
    for st, sr, so in fnames.values:
        if debug:
            print(f"Current file: ../{prefix}/{st}/{sr}/{so}.dcm")
        dicom = get_img(f"../{prefix}/{st}/{sr}/{so}.dcm")
        image = convert_to_rgb(dicom)
        X.append(image)
        del st, sr, so
        if len(X) == batch_size:
            if test:
                yield np.array(X)
                del X
            else:
                yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
                del X
            gc.collect()
            X = []
            batch += 1
    if test:
        yield np.array(X)
    else:
        yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
        del Y
    del X
    gc.collect()
    return

In [42]:
history = {}
start = time.time()
debug = 0
batch_size =1000
train_size = int(batch_size*0.9)
max_train_time = 3600*2 #4 hours
checkpoint = MC(filepath='../working/pe_detection_model.h1',monitor='val_loss', save_best_only=True, verbose=1)
#Train loop
for n, (x, y) in enumerate(custom_dcom_image_generator(batch_size, train.sample(frac=1), False, debug)):
    if len(x) < 10: #filter out empty data
        break
    clear_output(wait=True)
    print("Training batch: %i - %i" %(batch_size*n, batch_size*(n+1)))
    model = load_model('../working/pe_detection_model.h1')
    hist = model.fit(
        x[:train_size], 
        {'\n pe_present_on_image':y[:train_size, 0],
         '\n rv_lv_ratio_gte_1':y[:train_size, 1],
         '\n rv_lv_ratio_lt_1':y[:train_size, 2],
         '\n leftsided_pe':y[:train_size, 3],
         '\n chronic_pe':y[:train_size, 4],
         '\n rightsided_pe':y[:train_size, 5],
         '\n acute_and_chronic_pe':y[:train_size, 6],
         '\n central_pe':y[:train_size, 7],
         '\n indeterminate':y[:train_size, 8]},
        callbacks = checkpoint,
        validation_split=0.2,
        epochs=8,
        batch_size=15,
        verbose=debug
    )
    print("Metrics for batch validation:")
    model.evaluate(x[train_size:],
                   {'pe_present_on_image':y[train_size:, 0],
                    'rv_lv_ratio_gte_1':y[train_size:, 1],
                    'rv_lv_ratio_lt_1':y[train_size:, 2],
                    'leftsided_pe':y[train_size:, 3],
                    'chronic_pe':y[train_size:, 4],
                    'rightsided_pe':y[train_size:, 5],
                    'acute_and_chronic_pe':y[train_size:, 6],
                    'central_pe':y[train_size:, 7],
                    'indeterminate':y[train_size:, 8]
                   }
                  )
    try:
        for key in hist.history.keys():
            history[key] = np.concatenate([history[key], hist.history[key]], axis=0)
    except:
        for key in hist.history.keys():
            history[key] = hist.history[key]
    if time.time() - start >= max_train_time:
        print("Time's up!")
        break
  
    model.save('pe_detection_mode.h1')
    del model, x, y
    gc.collect()

In [43]:
for key in history.keys():
    if key.startswith('val'):
        continue
    else:
        epoch = range(len(history[key]))
        plt.plot(epoch, history[key]) #X=epoch, Y=value
        plt.plot(epoch,history['val_'+key])
        plt.title(key)
        if 'accuracy' in key:
            plt.axis([0,len(history[key]), -0.1, 1.1]) #Xmin, Xmax, Ymin, Ymax
        plt.legend(['train', 'test'], loc='lower right')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.show()
        plt.savefig("acc.jpg")

In [44]:
predictions = {}
stopper = 600*4 #4 hours
pred_start_time = time.time()

p, c = time.time(), time.time()
batch_size = 500
    
l = 0
n = test.shape[0]

for x in custom_dcom_image_generator(batch_size, test, True, False):
    clear_output(wait=True)
    model = load_model("../working/pe_detection_model.h1")
    preds = model.predict(x, batch_size=8, verbose=1)
    
    try:
        for key in preds.keys():
            predictions[key] += preds[key].flatten().tolist()
            
    except Exception as e:
        print(e)
        for key in preds.keys():
            predictions[key] = preds[key].flatten().tolist()
            
    l = (l+batch_size)%n
    print('Total predicted:', len(predictions['indeterminate']),'/', n)
    p, c = c, time.time()
    print("One batch time: %.2f seconds" %(c-p))
    print("ETA: %.2f" %((n-l)*(c-p)/batch_size))
    
    if c - pred_start_time >= stopper:
        print("Time's up!")
        break
    
    del model
    K.clear_session()
    
    del x, preds
    gc.collect()

In [45]:
for key in predictions.keys():
    print(key, np.array(predictions[key]).shape)

In [48]:
test_ids = []
for v in test.StudyInstanceUID:
    if v not in test_ids:
        test_ids.append(v)
        
test_preds = test.copy()
test_preds = pd.concat([test_preds, pd.DataFrame(predictions)], axis=1)
test_preds.to_csv('test_predictions.csv', index=False)
test_preds.head(50000)

In [30]:
sns.lmplot(x="pe_present_on_image",
           y="rv_lv_ratio_gte_1", 
           hue="chronic_pe",
           data=test_preds)

In [31]:
sns.scatterplot(x=test_preds['rv_lv_ratio_gte_1'], 
                y=test_preds['rv_lv_ratio_lt_1'], 
                hue=test_preds['acute_and_chronic_pe'])

In [ ]:
sns.scatterplot(x=test_preds['leftsided_pe'], y=test_preds['rightsided_pe'])

In [ ]:
test_preds.info()

In [ ]:
sns.countplot(test_preds['pe_present_on_image'])
xlabel ='pe present on image'
ylabel='count'

In [ ]:
sns.countplot(test_preds['rv_lv_ratio_gte_1'])
xlabel ='rv_lv_ratio_gte_1'
ylabel='count'

In [ ]:
sns.countplot(test_preds['central_pe'])
xlabel ='central_pe'
ylabel='count'